In [ ]:
# Transformers installation
!pip install transformers gdown pandas
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.0 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
import numpy as np
import evaluate
import gdown
import pandas as pd

from evaluate import evaluator
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, pipeline
from google.colab import drive

## Loading dataset

In [ ]:
url = "https://drive.google.com/file/d/1klgcUpRhAwhxnxHTWGhRJrAh3n_ZButf/view?usp=share_link"
output = "treino_tweets.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

url2 = "https://drive.google.com/file/d/1386svaKYa2_b3zPLBjmOzIVbOLH4ZurY/view?usp=sharing"
output2 = "test_tweets.csv"
gdown.download(url2, output2, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1klgcUpRhAwhxnxHTWGhRJrAh3n_ZButf
To: /content/treino_tweets.csv
100%|██████████| 1.22M/1.22M [00:00<00:00, 133MB/s]
Downloading...
From: https://drive.google.com/uc?id=1386svaKYa2_b3zPLBjmOzIVbOLH4ZurY
To: /content/test_tweets.csv
100%|██████████| 37.6k/37.6k [00:00<00:00, 50.8MB/s]


'test_tweets.csv'

In [ ]:
#Test
from datasets import Dataset, DatasetDict

# train_dataset = load_dataset("csv", data_files='/content/treino_tweets.csv', delimiter='\t')
# test_dataset = load_dataset("csv", data_files='/content/test_tweets.csv', delimiter='\t')

train_dataset = pd.read_csv('/content/treino_tweets.csv', delimiter = '\t')
test_dataset = pd.read_csv('/content/test_tweets.csv', delimiter = '\t')

# tdf = pd.DataFrame({"a": [1, 2, 3], "b": ['hello', 'ola', 'thammi']})
# vdf = pd.DataFrame({"a": [4, 5, 6], "b": ['four', 'five', 'six']})

tdf = pd.DataFrame(train_dataset)
vdf = pd.DataFrame(test_dataset)
tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)
ds = DatasetDict()
ds['train'] = tds
ds['validation'] = vds
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'prediction'],
        num_rows: 15212
    })
    validation: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 300
    })
})


## Preprocess

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("PORTULAN/albertina-ptbr-base")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_train_dataset = ds['train'].map(preprocess_function, batched=True)
tokenized_test_dataset = ds['validation'].map(preprocess_function, batched=True)

Map:   0%|          | 0/15212 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset.rename_columns({'prediction' : 'labels'})
tokenized_train_dataset = tokenized_train_dataset.rename_columns({'prediction' : 'labels'})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [ ]:
accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
eval_comb = evaluate.combine(["accuracy", "f1", "precision", "recall"])

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Training

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "PORTULAN/albertina-ptbr-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at PORTULAN/albertina-ptbr were not used when initializing DebertaV2ForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at P

In [ ]:
training_args = TrainingArguments(
    output_dir="Claudiobot",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16=True,
    label_names = ["labels"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 26>:26                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2759 in training_step            │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2784 in compute_loss             │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")                                                 │
│   2782 │   │   else:                                                                             │
│   2783 │   │   │   labels = None                                                                 │
│ ❱ 2784 │   │   outputs = model(**inputs)                                                         │
│   2785 │   │   # Save past state if it exists              

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/AlbertinaModel")

## Inference

In [ ]:
classifier = pipeline("text-classification", model="/content/gdrive/MyDrive/AlbertinaModel")

##Results


In [ ]:
tokenized_test_dataset

In [ ]:
task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=classifier,
    data=tokenized_test_dataset,
    metric=eval_comb,
    label_mapping={"NEGATIVE": 0, "POSITIVE": 1},
    label_column = 'label',
)

print(eval_results)